## Process SG Coarse Dataset and Digest ONET CIP and SOC ID

In [95]:
!pip install pandas sklearn
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [96]:
# Import the necessary packages
import pandas as pd
# from sklearn.metrics import jaccard_score
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet
# from nltk.metrics import jaccard_distance
import re
from operator import itemgetter


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [97]:
# Load the data into a pandas dataframe:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [98]:
# Note: You will need to create a shortcut for the "MTECH IS Project" shared folder on your MyDrive to be able to access it directly.
!ls /content/drive/MyDrive/MTECH_IS_Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv
!ls /content/drive/MyDrive/MTECH_IS_Project/data/Education/CIP2020_instructional_programs.csv
!ls /content/drive/MyDrive/MTECH_IS_Project/data/Education/CIP2020_SOC2018.csv

/content/drive/MyDrive/MTECH_IS_Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv
/content/drive/MyDrive/MTECH_IS_Project/data/Education/CIP2020_instructional_programs.csv
/content/drive/MyDrive/MTECH_IS_Project/data/Education/CIP2020_SOC2018.csv


In [99]:
# load the SG Coarse Dataset
file = "/content/drive/MyDrive/MTECH_IS_Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv"
# file = "/content/drive/MyDrive/MTECH\ IS\ Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv"
sg_df = pd.read_csv(file, encoding = 'unicode_escape', engine ='python')
# sg_df = pd.read_csv(file, engine ='python')
sg_df.head(3)

,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile
0,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy and Business,97.4,96.1,3701,3200,3727,3350,2900,4000
1,2013,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy (3-yr direct Honours Programme),97.1,95.7,2850,2700,2938,2700,2700,2900
2,2013,Nanyang Technological University,College of Business (Nanyang Business School),Business (3-yr direct Honours Programme),90.9,85.7,3053,3000,3214,3000,2700,3500


In [100]:
len(sg_df['degree'].unique())

337

In [101]:
# load the Onet Coarse Instructural Program (CIP) Dataset
# file = "/content/drive/My Drive/data_processing/CIP2020_instructional_programs.csv"
file = "/content/drive/MyDrive/MTECH_IS_Project/data/Education/CIP2020_instructional_programs.csv"
cip_df = pd.read_csv(file,encoding = 'utf-8')

index_col = "CIP2020Code"

cip_df.head(3)

,CIP2020Code,CIP2020Title
0,1.0000,"Agriculture, General."
1,1.0101,"Agricultural Business and Management, General."
2,1.0102,Agribusiness/Agricultural Business Operations.


In [102]:
# load the CIP to SOC equivalent
# file = "/content/drive/My Drive/data_processing/CIP2020_SOC2018.csv"
file = "/content/drive/MyDrive/MTECH_IS_Project/data/Education/CIP2020_SOC2018.csv"
soc_df = pd.read_csv(file,encoding = 'utf-8')
soc_df.head(3)
#soc_df.head()
# soc_df.columns = ['CIP2020Code', 'SOC2018Code']
# soc_df.head()

,CIP2020Code,SOC2018Code
0,1.0,19-1011
1,1.0,19-1012
2,1.0,19-1013


In [103]:
# Merging Onet Education Dataset
onet_soc_cip_df = cip_df.merge(
  soc_df, on=index_col, how="left"
)

onet_soc_cip_df.head(10)

,CIP2020Code,CIP2020Title,SOC2018Code
0,1.0000,"Agriculture, General.",19-1011
1,1.0000,"Agriculture, General.",19-1012
2,1.0000,"Agriculture, General.",19-1013
3,1.0000,"Agriculture, General.",19-4012
4,1.0000,"Agriculture, General.",25-1041
5,1.0101,"Agricultural Business and Management, General.",11-9013
6,1.0101,"Agricultural Business and Management, General.",25-1041
7,1.0101,"Agricultural Business and Management, General.",45-1011
8,1.0102,Agribusiness/Agricultural Business Operations.,11-9013
9,1.0102,Agribusiness/Agricultural Business Operations.,25-1041


In [104]:
# Function Declaration

#Lower Case
def tolower_case(text):
    text = text.lower()
    #text=re.sub('[^a-z]+', ' ',text)
    return text

# Remove Punctuation
def punctuation_removal(text):
    # Remove the all the punctuations except '
    new_text = re.sub(',|!|\?|\"|<|>|\(|\)|\[|\]|\{|\}|@|#|\+|\=|\-|\_|~|\&|\*|\^|%|\||\$|/|`|\.|\'',
                          '', text,count=0, flags=0)
    return new_text

# Expand Abbreviation
def expand_abbreviation(text):
    replacement_patterns = [
        (r'won\'t', 'will not'),
        (r'can\'t', 'cannot'),
        (r'i\'m', 'i am'),
        (r'ain\'t', 'is not'),
        (r'(\w+)\'ll', '\g<1> will'),
        (r'(\w+)n\'t', '\g<1> not'),
        (r'(\w+)\'ve', '\g<1> have'),
        (r'(\w+)\'s', '\g<1> is'),
        (r'(\w+)\'re', '\g<1> are'),
        (r'(\w+)\'d', '\g<1> would')]
    patterns = [(re.compile(regex), repl) for (regex, repl) in replacement_patterns]

    new_text = text
    for (pattern, repl) in patterns:
        (new_text, count) = re.subn(pattern, repl, new_text)
    return new_text

# Tokenization

def tokenization(text):
    new_text = nltk.word_tokenize(text)
    return new_text

# Lemmatization

def get_wordnet_pos(word):
    pack = nltk.pos_tag([word])
    tag = pack[0][1]
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV

    else:
        return None

# Stopword Removal

def lemmatization(text):
    lemmatizer = nltk.stem.WordNetLemmatizer()

    new_text = [lemmatizer.lemmatize(word, get_wordnet_pos(word) or wordnet.NOUN) for word in text]

    return new_text

# find the total number of values in both sets

def Union(lst1, lst2):
    final_list = lst1 + lst2
    return final_list

# find the number of shared values between sets
def Intersection(lst1, lst2):
    return set(lst1).intersection(lst2)

def jaccard_similarity(shared, total):
    jaccard_index = len(shared) / len(total)
    return jaccard_index

In [105]:
# Text Processing
def text_preprocessing(raw_text):
    text = tolower_case(raw_text)
    text = expand_abbreviation(text)
    text = punctuation_removal(text)
    text = tokenization(text)
    #text = stopword_removal(text)
    text = lemmatization(text)
    return text

In [106]:
degree_title = text_preprocessing(sg_df['degree'][38])
#cip_title = text_preprocessing(cip_df['CIP2020Title'][2])
cip_title = text_preprocessing(onet_soc_cip_df['CIP2020Title'][5])
print("Degree_Name:", degree_title)
print("CIP_Title:", cip_title)

# https://towardsdatascience.com/semantic-search-measuring-meaning-from-jaccard-to-bert-a5aca61fc325
# find the number of shared values between sets
shared = Intersection(cip_title, degree_title)
print("Intersecction of Set:", shared)

# find the total number of values in both sets
total = Union(cip_title, degree_title)
print("Union of Set:",total)

# calculate Jaccard similarity  
# jac_sim = len(shared) / len(total)
jaccard_sim = jaccard_similarity(shared, total)
print("Jaccard Simmilary::",jaccard_sim)



Degree_Name: ['bachelor', 'of', 'business', 'administration', 'hons']
CIP_Title: ['agricultural', 'business', 'and', 'management', 'general']
Intersecction of Set: {'business'}
Union of Set: ['agricultural', 'business', 'and', 'management', 'general', 'bachelor', 'of', 'business', 'administration', 'hons']
Jaccard Simmilary:: 0.1


In [107]:
# construct a new DataFrame 
new_df = pd.DataFrame(data=None, index=None, columns=sg_df.columns, dtype=None, copy=None)

new_df["CIP2020Code"] = ''
new_df["SOC2018Code"] = ''

print(new_df)

Empty DataFrame
Columns: [year, university, school, degree, employment_rate_overall, employment_rate_ft_perm, basic_monthly_mean, basic_monthly_median, gross_monthly_mean, gross_monthly_median, gross_mthly_25_percentile, gross_mthly_75_percentile, CIP2020Code, SOC2018Code]
Index: []


In [109]:
counter=len(sg_df['degree'])
print(counter)

1121


In [113]:
# find matching sg Coarse for each onet coarse(CIP Code)
max_match = 4
i = 0
counter=len(sg_df['degree'])
#print(counter)

for onet_cip_coarse in onet_soc_cip_df['CIP2020Title']:
    j=0
    #threshold = 0.3
    #threshold = 0.4
    #threshold = 0.5
    #threshold = 0.6
    threshold = 0.8
    sg_coarse_ids = []
    for sg_coarse_title in sg_df['degree']:
        degree_title = text_preprocessing(sg_coarse_title)
        cip_title = text_preprocessing(onet_cip_coarse)
        if (len(cip_title) == 0) or (len(degree_title) == 0):
            continue

        # JACCARD SIMILARITY COMPUTATION
        # REFERENCE: https://towardsdatascience.com/semantic-search-measuring-meaning-from-jaccard-to-bert-a5aca61fc325

        # find the number of shared values between sets
        shared = Intersection(cip_title, degree_title)
        # find the total number of values in both sets
        total = Union(cip_title, degree_title)
        # and calculate Jaccard similarity  
        jaccard_sim = jaccard_similarity(shared, total)
        #jac_sim = len(shared) / len(total)
        # print(jaccard_sim)

        if jaccard_sim > threshold:
            # if found a match, append to new dataframe
            if (len(sg_coarse_ids) == max_match):
                sg_coarse_ids = sorted(sg_coarse_ids, key=itemgetter(1))
                threshold = sg_coarse_ids[0][1]
            if (len(sg_coarse_ids) > max_match):
                # if jaccard_sim is higher than any of the existing one, replace the lowest jaccard_sim
                sg_coarse_ids = sorted(sg_coarse_ids, key=itemgetter(1))
                sg_coarse_ids[0] = [j, jaccard_sim]
                threshold = sg_coarse_ids[0][1]   
            else:
                sg_coarse_ids.append([j, jaccard_sim])
        j+=1  
        
    # add to coarse list
    #print(sg_coarse_ids)
    for sg_coarse_id in sg_coarse_ids:
        new_df.loc[len(new_df)] = sg_df.loc[i].tolist() + onet_soc_cip_df[['CIP2020Code', 'SOC2018Code']].loc[sg_coarse_id[0]].tolist()

    i+=1
    if (i <= counter):
        print(i, "Processing: ",sg_coarse_ids)
    else:
        print("### END of SG DATASET ###")
        break
    # if (i > 5):
    #     break
new_df.head(10)  

1 Processing:  []
2 Processing:  []
3 Processing:  []
4 Processing:  []
5 Processing:  []
6 Processing:  []
7 Processing:  []
8 Processing:  []
9 Processing:  []
10 Processing:  []
11 Processing:  []
12 Processing:  []
13 Processing:  []
14 Processing:  []
15 Processing:  []
16 Processing:  []
17 Processing:  []
18 Processing:  []
19 Processing:  []
20 Processing:  []
21 Processing:  []
22 Processing:  []
23 Processing:  []
24 Processing:  []
25 Processing:  []
26 Processing:  []
27 Processing:  []
28 Processing:  []
29 Processing:  []
30 Processing:  []
31 Processing:  []
32 Processing:  []
33 Processing:  []
34 Processing:  []
35 Processing:  []
36 Processing:  []
37 Processing:  []
38 Processing:  []
39 Processing:  []
40 Processing:  []
41 Processing:  []
42 Processing:  []
43 Processing:  []
44 Processing:  []
45 Processing:  []
46 Processing:  []
47 Processing:  []
48 Processing:  []
49 Processing:  []
50 Processing:  []
51 Processing:  []
52 Processing:  []
53 Processing:  []
54

,year,university,school,degree,employment_rate_overall,employment_rate_ft_perm,basic_monthly_mean,basic_monthly_median,gross_monthly_mean,gross_monthly_median,gross_mthly_25_percentile,gross_mthly_75_percentile,CIP2020Code,SOC2018Code
0,2013,Nanyang Technological University,College of Engineering,Electrical and Electronic Engineering,91,88.7,3133,3000,3187,3100,3000,3440,1.0105,41-4012
1,2013,Nanyang Technological University,College of Engineering,Electrical and Electronic Engineering,91,88.7,3133,3000,3187,3100,3000,3440,1.0605,37-1012
2,2013,Nanyang Technological University,College of Engineering,Electrical and Electronic Engineering,91,88.7,3133,3000,3187,3100,3000,3440,1.1199,19-1013
3,2013,Nanyang Technological University,College of Engineering,Electrical and Electronic Engineering,91,88.7,3133,3000,3187,3100,3000,3440,1.8204,43-1011
4,2013,Nanyang Technological University,College of Engineering,Electrical and Electronic Engineering,91,88.7,3133,3000,3187,3100,3000,3440,3.0508,25-1043
5,2013,Nanyang Technological University,College of Engineering,Environmental Engineering,84,81.5,3091,3000,3140,3000,3000,3300,1.0105,41-4012
6,2013,Nanyang Technological University,College of Engineering,Environmental Engineering,84,81.5,3091,3000,3140,3000,3000,3300,1.0605,37-1012
7,2013,Nanyang Technological University,College of Engineering,Environmental Engineering,84,81.5,3091,3000,3140,3000,3000,3300,1.1199,19-1013
8,2013,Nanyang Technological University,College of Engineering,Environmental Engineering,84,81.5,3091,3000,3140,3000,3000,3300,1.8204,43-1011
9,2013,Nanyang Technological University,College of Engineering,Environmental Engineering,84,81.5,3091,3000,3140,3000,3000,3300,3.0508,25-1043


In [114]:
# Create a CSV and store it on the Project Share Drive
new_df.to_csv("/content/drive/MyDrive/MTECH_IS_Project/data/sg_coarse_to_onet_th8.csv",  encoding='utf-8')

In [112]:
len(new_df['degree'].unique())

42